<a href="https://colab.research.google.com/github/joaoantoniopc/projeto_1/blob/main/desafio_detalhamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color = blue> Pipeline do



1.   Entendimento do Problema de Negócio
2.   Análise Exploratória de Dados
3.   Processamento dos Dados
4.   Análise Exploratória de Dados
5.   Análise Exploratória de Dados








## ![](https://miro.medium.com/v2/resize:fit:679/0*Yb_BsikIKFAtuKj9.gif)



#1° Entendimento do Problema de Negócio




    1. Fontes de Dados Simuladas: Crie duas fontes de dados simuladas em formato CSV: uma representando dados de vendas (com campos como data, produto, valor) e outra representando dados de clientes (com campos como ID, nome, email).

    2. Transformação de Dados: Desenvolva um processo de transformação de dados usando a linguagem de programação de sua escolha (por exemplo, Python ou SQL) para combinar os dados de vendas e clientes com base no ID do cliente. O resultado deve ser uma tabela consolidada com informações relevantes para análise.

    3. Data Warehouse Simulado: Modele um esquema simples para um Data Warehouse que possa acomodar os dados consolidados. Pode ser um esquema estrela básico com tabelas de fatos e dimensões.

    4. Carga no Data Warehouse: Crie um script ou processo para carregar os dados consolidados na estrutura do Data Warehouse simulado.

    5. Visualização de Dados: Use uma ferramenta de visualização de dados de sua escolha (por exemplo, Power BI, Tableau ou outra) para criar um painel simples que apresente informações sobre as vendas. Você pode incluir gráficos de vendas por período, produto mais vendido etc.

#2° Fontes de Dados Simuladas e transformações

Em anexo consta os arquivos sql criados para extração dos cliente onde já foram realizadas as transformações de normalização de dados(extract_cliente.sql) e extração das vendas(extract_vendas.sql).

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#2° Análise Exploratória dos Dados

Neste passo foi realizado merge dos arquivos e removido campos duplicados.

Instalacao das bibliotecas

In [5]:
!pip install pandas   gdown
!pip install sweetviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 36.3 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import io
import sweetviz as sv
import gdown

# Lendo os DataFrames com o delimitador correto (vírgula) e indicando que a primeira linha contém cabeçalhos

# Carregando os dados do arquivo CSV 'clientes.csv' em clientes_df
clientes_df = pd.read_csv("clientes.csv",  header=0, encoding='utf-8')

# Carregando os dados do arquivo CSV 'vendas.csv' em vendas_df
vendas_df = pd.read_csv("vendas.csv",  header=0, encoding='utf-8')

# Realizando a junção dos DataFrames pelos campos "id do cliente"
resultado_df = pd.merge(clientes_df, vendas_df, on="id")

# Removendo colunas duplicadas após o merge
resultado_df = resultado_df.loc[:, ~resultado_df.columns.duplicated()]

# Realizando a análise com Sweetviz no DataFrame resultado_df
resultado = sv.analyze(resultado_df)

# Gerando um relatório HTML chamado 'SWEETVIZ_REPORT.html'
resultado.show_html('SWEETVIZ_REPORT.html')

> Neste momento foi gerado o arquivo de análise exploratória dos dados: SWEETVIZ_REPORT.html

# 3° Processamento de Dados

### Importação das bibliotecas

In [ ]:
# Manipulação de Dados
import pandas as pd
import numpy as np

# Bibliotecas de IA - Machine Learning e Automações
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, accuracy_score
import pickle

### Amostragem dos dados

In [ ]:
# Separar features e target
X = df.drop('prevendas', axis=1)
y = df['prevendas']

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Engenharia de Atributos

In [ ]:
# Listar as colunas numéricas e categóricas
numeric_features = X.select_dtypes(include=[np.number]).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Criar os transformers para pré-processamento
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(sparse=False)

# Criar o ColumnTransformer para aplicar as transformações nas colunas apropriadas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Aplicar as transformações no preprocessor
X_train_preprocessed = preprocessor.fit_transform(X_train)
numeric_columns = numeric_features.tolist()
categorical_columns = preprocessor.transformers_[1][1].get_feature_names_out(categorical_features)
all_columns = numeric_columns + list(categorical_columns)
X_train_preprocessed = pd.DataFrame(X_train_preprocessed, columns=all_columns)



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Cria Dados Sintéticos para resolver o problema de Desbalanceamento do Target
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train_preprocessed, y_train)



#4° Construção e Avaliação da IA Preditiva de Vendas

### Escolha do Algoritmo de Machine Learning e Treinamento com os Dados

In [ ]:
# Escolher um algoritmo de aprendizado de máquina
model = LGBMClassifier(n_estimators=200)

# Treinar o modelo no conjunto de treinamento
pipeline = make_pipeline(model)
pipeline.fit(X_train_resampled, y_train_resampled)

# Aplicar as mesmas transformações no conjunto de teste
X_test_preprocessed = preprocessor.transform(X_test)
X_test_preprocessed = pd.DataFrame(X_test_preprocessed, columns=all_columns)



In [ ]:
# Avaliar o modelo no conjunto de teste
y_pred = pipeline.predict(X_test_preprocessed)

# Métricas de avaliação
print('Acurácia:', accuracy_score(y_test, y_pred))
print('\nClassification Report:\n', classification_report(y_test, y_pred))




Acurácia: 0.910206789782152

Classification Report:
               precision    recall  f1-score   support

         não       0.95      0.95      0.95      7952
         sim       0.64      0.60      0.62      1091

    accuracy                           0.91      9043
   macro avg       0.79      0.77      0.78      9043
weighted avg       0.91      0.91      0.91      9043



### Lista das Variáveis mais Importantes da IA

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Obter a importância das variáveis
importance = pipeline.named_steps['lgbmclassifier'].feature_importances_
feature_names = all_columns

# Criar um DataFrame com a importância das variáveis
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importance})

# Ordenar o DataFrame pela importância das variáveis em ordem decrescente
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Criar o gráfico de barras da importância das variáveis usando o Plotly
fig = go.Figure(data=[go.Bar(x=importance_df['Importance'], y=importance_df['Feature'], orientation='h')])
fig.update_layout(
    title='Importância das Variáveis',
    xaxis_title='Importância',
    yaxis_title='Variáveis'
)
# Inverta a ordem do eixo y para que o recurso mais importante esteja no topo
fig.update_yaxes(autorange="reversed")

fig.show()

### Salvamento da IA

In [ ]:
# Salvar o modelo treinado, o pré-processador e os nomes das colunas em um arquivo
with open('model.pkl', 'wb') as f:
    pickle.dump({
        'model': pipeline,
        'preprocessor': preprocessor,
        'columns': all_columns
    }, f)

![](https://medicinasa.com.br/wp-content/uploads/2021/08/pulmao-artificial-digital-imagem.gif)

#**5° Implementação da IA Preditiva de Vendas - Deploy**

### Avaliação da IA com as Métricas

## Acessar o **ngrok**
https://ngrok.com/

![](https://cienciadosdados.com/images/2023/06/ngrok.png)
 **O ngrok** é uma ferramenta que permite criar um túnel seguro entre sua máquina local e a internet

 Ele não é uma solução de hospedagem permanente para implantar um aplicativo.

 É usado principalmente como uma ferramenta de desenvolvimento para fins de compartilhamento temporário.


###**Passo-a-Passo**

1° Passo: Download do Ngrok

2° Passo: Instalação

3° Acessar o Token

In [ ]:
#Instalação das Bibliotecas
!pip install streamlit
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##### Criação do script do app no streamlit

In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle

def load_model():
    with open('model.pkl', 'rb') as f:
        data = pickle.load(f)
    model = data['model']
    preprocessor = data['preprocessor']
    all_columns = data['columns']
    return model, preprocessor, all_columns

def predict(model, preprocessor, inputs, all_columns):
    df = pd.DataFrame([inputs])
    df = pd.DataFrame(preprocessor.transform(df), columns=all_columns)
    prediction = model.predict(df)
    return prediction

model, preprocessor, all_columns = load_model()

print(model)
print("-------")
print(preprocessor)
print("-------")
print(all_columns)

#sys.exit(0)

st.title('IA de Previsão de Vendas')
st.write('Insira os dados abaixo:')

inputs = {
    'idade': st.slider('Idade', min_value=18, max_value=100, value=40),
    'emprego': st.selectbox('Profissão', options=['gerente', 'técnico', 'empreendedor', 'trabalhador-manual', 'desconhecido', 'aposentado', 'administrativo', 'serviços', 'autônomo', 'desempregado', 'doméstica', 'estudante']),
    'estado_civil': st.selectbox('Estado Civil', options=['casado', 'solteiro', 'divorciado']),
    'educacao': st.selectbox('Educação', options=['terciário', 'secundário', 'desconhecido', 'primário']),
    'padrao': st.selectbox('Inadimplente', options=['não', 'sim']),
    'saldo': st.number_input('Saldo', min_value=-10000, max_value=100000, value=0),
    'moradia': st.selectbox('Empréstimo Imobiliário', options=['sim', 'não']),
    'emprestimo': st.selectbox('Empréstimo Pessoal', options=['sim', 'não']),
    'contato': st.selectbox('Contato', options=['desconhecido', 'celular', 'telefone']),
    'dia': st.slider('Dia', min_value=1, max_value=31, value=15),
    'mes': st.selectbox('Mês', options=['maio', 'junho', 'julho', 'agosto', 'outubro', 'novembro', 'dezembro', 'janeiro', 'fevereiro', 'março', 'abril', 'setembro']),
    'duracao': st.slider('Duração', min_value=1, max_value=5000, value=1000),
    'campanha': st.slider('Campanha', min_value=1, max_value=100, value=10),
    'dias_passados': st.slider('Dias Passados', min_value=-1, max_value=1000, value=-1),
    'anterior': st.slider('Anterior', min_value=0, max_value=100, value=0),
    'resultado_anterior': st.selectbox('Resultado Anterior', options=['desconhecido', 'fracasso', 'outro', 'sucesso'])
}

if st.button('Predict'):
    prediction = predict(model, preprocessor, inputs, all_columns)
    st.write('A previsão do modelo é:', 'sim' if prediction[0] == 1 else 'não')



Overwriting streamlit_app.py


In [ ]:
# Token de segurança do Ngrok

#original que já estava digitaeo
#!ngrok authtoken "2PsXeEYy7oOV5hgnF3UrRdDHwOB_5gmKKsHz23KdJHwEbtLWL"

#token que eu inclui, mas deu erro.
#!ngrok authtoken "2RcKuV5iQ18V3kmyfa3IP78OBDe_2d9jEyzh7VMMDE9M88tEV"

#token, apos o erro pesquisado no chatgpt e regerado o token e substituído aqui
!ngrok authtoken "2Rc9Nc6ZnATOwiUzPONQAm3Rss9_5Xjg23MetKXXBm7Kg2dYS"


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# Execução do streamlit no Colab
!streamlit run streamlit_app.py &>/dev/null&


In [ ]:
# Criação da Porta de Conexão da sua máquina e a Internet
from pyngrok import ngrok
public_url = ngrok.connect(port='8501')
print(public_url)

!streamlit run --server.port 80 streamlit_app.py >/dev/null

http://87b2-35-185-199-73.ngrok-free.app


#Abaixo estão alguns testes que eu fiz, para verificar se encontro alguma condição com previsão = "sim".



In [ ]:
prev_sim = df.loc[(df['prevendas'] == 'sim')]

prev_sim.head(10)

#eu acho que não dá certo, pois podem ser que não estão no arquivo y_train.


,idade,emprego,estado_civil,educacao,padrao,saldo,moradia,emprestimo,contato,dia,mes,duracao,campanha,dias_passados,anterior,resultado_anterior,prevendas
83,59,administrativo,casado,secundário,não,2343,sim,não,desconhecido,5,maio,1042,1,-1,0,desconhecido,sim
86,56,administrativo,casado,secundário,não,45,não,não,desconhecido,5,maio,1467,1,-1,0,desconhecido,sim
87,41,técnico,casado,secundário,não,1270,sim,não,desconhecido,5,maio,1389,1,-1,0,desconhecido,sim
129,55,serviços,casado,secundário,não,2476,sim,não,desconhecido,5,maio,579,1,-1,0,desconhecido,sim
168,54,administrativo,casado,terciário,não,184,não,não,desconhecido,5,maio,673,2,-1,0,desconhecido,sim
270,42,gerente,solteiro,terciário,não,0,sim,sim,desconhecido,5,maio,562,2,-1,0,desconhecido,sim
388,56,gerente,casado,terciário,não,830,sim,sim,desconhecido,6,maio,1201,1,-1,0,desconhecido,sim
390,60,aposentado,divorciado,secundário,não,545,sim,não,desconhecido,6,maio,1030,1,-1,0,desconhecido,sim
446,39,técnico,solteiro,desconhecido,não,45248,sim,não,desconhecido,6,maio,1623,1,-1,0,desconhecido,sim
457,37,técnico,casado,secundário,não,1,sim,não,desconhecido,6,maio,608,1,-1,0,desconhecido,sim


#copiei o resultado neste arquivo TEXTO.


idade	emprego	estado_civil	educacao	padrao	saldo	moradia	emprestimo	contato	dia	mes	duracao	campanha	dias_passados	anterior	resultado_anterior	prevendas
83	59	administrativo	casado	secundário	não	2343	sim	não	desconhecido	5	maio	1042	1	-1	0	desconhecido	sim

86	56	administrativo	casado	secundário	não	45	não	não	desconhecido	5	maio	1467	1	-1	0	desconhecido	sim

87	41	técnico	casado	secundário	não	1270	sim	não	desconhecido	5	maio	1389	1	-1	0	desconhecido	sim

129	55	serviços	casado	secundário	não	2476	sim	não	desconhecido	5	maio	579	1	-1	0	desconhecido	sim

168	54	administrativo

In [ ]:
# irei selecionar algum registro com prevendas = "sim".
# usarei o registro 32.192

print(y_train)

3344     não
17965    não
18299    não
10221    não
32192    sim
        ... 
11284    sim
44732    não
38158    não
860      não
15795    não
Name: prevendas, Length: 36168, dtype: object


print(y_train)

# copiei o resultado neste arquivo TEXTO:


3344     não
17965    não
18299    não
10221    não
32192    sim
        ...
11284    sim
44732    não
38158    não
860      não
15795    não
Name: prevendas, Length: 36168, dtype: object

In [ ]:
# selecionei o registro 32.192 do arquivo X_train. Digitei no aplicativo Ia de Previsão de Vendas.
print("###")
X_train.iloc[32192]

###


idade                           52
emprego                   serviços
estado_civil                casado
educacao                 terciário
padrao                         não
saldo                         3094
moradia                        sim
emprestimo                     não
contato                    celular
dia                             17
mes                       novembro
duracao                         96
campanha                         1
dias_passados                   -1
anterior                         0
resultado_anterior    desconhecido
Name: 24335, dtype: object



print("###")
X_train.iloc[32192]

# copiei o resultado neste arquivo TEXTO:
###
idade                           52
emprego                   serviços
estado_civil                casado
educacao                 terciário
padrao                         não
saldo                         3094
moradia                        sim
emprestimo                     não
contato                    celular
dia                             17
mes                       novembro
duracao                         96
campanha                         1
dias_passados                   -1
anterior                         0
resultado_anterior    desconhecido
Name: 24335, dtype: object

In [ ]:
,

##### É só o começo...
##### Simbóra!